In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [3]:
# Read the .csv file
df_chrysler = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\chrysler.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_2000\2180083355.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_chrysler = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\chrysler.csv")


In [4]:
# Check the cols available
df_chrysler.columns

Index(['Contlr', 'ModuleType SBECIII', 'Generic Hardware P/N',
       'Latest Software', 'Older Software P/N Versions', 'Year', 'Body',
       'Engine', 'Software Description', 'Trans Type',
       'Sales Codes\neng trans emissions', 'VIN Example'],
      dtype='object')

In [5]:
# Map to rename Hardware col
map_dict = {
    "Generic Hardware P/N": "Hardware"
}

In [6]:
# Rename col Generic Hardware P/N to Hardware
df_chrysler_rn_hdw_col = df_chrysler.rename(columns=map_dict)

In [7]:
# Get the index with empty rows
df_chrysler_empty_rows_index = df_chrysler_rn_hdw_col[df_chrysler_rn_hdw_col["Contlr"].isna() == True].index

In [8]:
# Drop the empty rows
df_chrysler_drop_empty = df_chrysler_rn_hdw_col.drop(df_chrysler_empty_rows_index)

In [9]:
# Drop the repeated columns
df_chrysler_repeated_cols_index = df_chrysler_drop_empty[df_chrysler_drop_empty["Contlr"] == "Contlr"].index

In [10]:
# Drop the repeated cols
df_chrysler_drop_empty = df_chrysler_drop_empty.drop(df_chrysler_repeated_cols_index)

# Cols to keep:
- Hardware
- Latest Software
- Older Software P/N Versions

In [11]:
# List with cols to keep
cols_to_keep = ["Hardware", "Latest Software", "Older Software P/N Versions"]

In [12]:
# Create a df with only the cols to keep
df_chrysler_w_cols_to_keep = df_chrysler_drop_empty[cols_to_keep]

In [13]:
# Function to fill out null rows
def fill_null_rows(df):
    # Loop to iterate over all cols
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")
    return df

In [14]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [15]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [16]:
# Call the function to fill out null rows
df_chrysler_fill_nullValues = fill_null_rows(df_chrysler_w_cols_to_keep)

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_2000\132036243.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna("Not Available")


In [17]:
# Show the cols
df_chrysler_fill_nullValues.columns

Index(['Hardware', 'Latest Software', 'Older Software P/N Versions'], dtype='object')

### Call the function to the explode all items of the cols

In [18]:
df_chrysler_expl_hdw_col = split_part_numbers(df_chrysler_fill_nullValues, "Hardware")

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_2000\3366128590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


In [19]:
df_chrysler_expl_lastSftw_col = split_part_numbers(df_chrysler_expl_hdw_col, "Latest Software")

In [20]:
df_chrysler_expl_old_sftw_col = split_part_numbers(df_chrysler_expl_lastSftw_col, "Older Software P/N Versions")

In [21]:
pn = df_chrysler_expl_old_sftw_col[df_chrysler_expl_old_sftw_col['Older Software P/N Versions'] == ' 56040616AJ']

In [22]:
# Reset the index
df_chrysler_reset_index = df_chrysler_expl_old_sftw_col.reset_index(drop=True)

## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

### Next lines to rename the cols

In [23]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "Hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [24]:
# Call the function to rename the cols
df_chrysler_rn_cols = rename_col_names(df_chrysler_reset_index)

In [25]:
# Check info
df_chrysler_rn_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50945 entries, 0 to 50944
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hardware  50945 non-null  object
 1   pn2       50945 non-null  object
 2   pn1       50945 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
# Export to .csv file
df_chrysler_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-chrysler.csv", index=False)

## Database Import Documentation

After cleaning the data and preparing the DataFrame as needed, the data will be imported into a database. The functions below are intended to connect to the database and import the datasets.

In [27]:
def import_datasets_to_db(df, df_name=str):

    '''
    Import the df cleaned to the db

    Parameters: 
        df (DataFrame): main df to import to the db.
        df_name: string to label the df into the db.

    Returns:
        Import the df to the db. 
    '''
    # Setting up the connection with the PosgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

    return df.to_sql(df_name, engine, if_exists='replace', index=False)


In [28]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_chrysler_rn_cols, 'part_numbers_chrysler')

945